## 1.1
what is the result printed by each expression below?

`10`  
10

`(+ 5 3 4)`  
12

`(- 9 1)`  
8

`(/ 6 2)`  
3

`(+ (* 2 4) (- 4 6))`  
8 + -2  
6  

`(define a 3)`  
a (evaluates to 3)

`(define b (+ a 1))`  
b (evaluates to 4)

`(+ a b (* a b))`  
3 + 4 + (3 * 4)  
7 + 12  
19

`(= a b)`  
false (represented as `#f`)

`(if (and (> b a) (<b (* a b)))
    b
    a)`

(> b a) = ( > 4 3) = true  
( * a b) = 12  
(> 4 12) = false  
(if (and true false) a b)  
(if false a b)  
b  
4

`(cond ((= a 4) 6)
    ((= b 4) (+ 6 7 a))
    (else 25))`
note: a is 3, b is 4  
(+ 6 7 a)  
(+ 6 7 3)
16

`(+ 2 (if (> b a) b a ))`
6

`(* (cond ((> a b) a)
        ((< a b) b)
        (else -1))
    (+ a 1))`  
4 * a + 1  
4 * 3 + 1  
16

## 1.2
translate the following into prefix form:  
(5 + 4 + (2 - (3 - (6 + 4/5)))) / (3(6-2)(2-7))  
```mit-scheme
(/ (+ 5 4
      (- 2 (- 3 (+ 6 (/ 4 5)))))
   (* 3 (- 6 2) (- 2 7)))
```

In [13]:
(/ (+ 5 4
      (- 2 (- 3 (+ 6 (/ 4 5)))))
   (* 3 (- 6 2) (- 2 7)))


-37/150

## 1.3
Define a procedure that takes 3 numbers as args and returns the sum of the squares of the two larger numbers

In [36]:
(define (sq a) (* a a))
(display (sq 2))
(define (sumsq a b) 
  (+ (sq a) (sq b)))
(sumsq 6 3)
(define (dostuff a b c)
        (cond (and (< c b)(< c a)) (sumsq a b))
        (cond (and (< b a)(< b c)) (sumsq a c))
        (cond (and (< a b)(< a c)) (sumsq b c)))




45

## 1.4
functions are first class in lisp. knowing that, how does the following work?
```
(define (a-plus-abs-b a b)
    ((if (> b 0) + -) a b))
```
the `if` evaluates to + if b > 0. Otherwise it evaluates to -. that operation is applied to `a` and `b`. thus if b is negative, one subtracts b, which is the same as adding |b|. the absolute value operation doesn't need to exist as its own step

## 1.5
you can test if your interpreter is using applicative order or normal-order evaluation using the following:
```
(define (p) (p))
(define (test x y)
    (if (= x 0)
        0
        y))
(test 0 (p))
```
what happens in either case?
normal-order is lazy; it doesn't evaluate stuff until the value is needed. If we're doing applicative order, i think it hangs because `p` is only defined in terms of itself and should recurse infinitely. Scheme is applicative-order

since normal order is lazy, it doesn't need to actually check y until the `if` check fails, so it should kick out with a `0`

## 1.6  
why can't `if` just be done in terms of `cond`? let's look at the following implementation:

```
(define (new-if predicate then-clause else-clause)
  (cond (predicate then-clause)
        (else else-clause)))

(define (sq-iter guess x)
    (new-if (good-enough? guess x)
            guess
            (sq-iter (improve guess x)
                     x)))
```

As it turns out, lazy evaluation is important! cond doesn't short circuit, so sq-iter gets called forever

## 1.7
why does our current sqrt function not work well for large and small numbers?
* our `good-enough?` method won't work for `n` smaller than the threshold, since all `n` will fall in that threshold
* for very large numbers, no `n` will fall in the threshold (at least not for a very very long time)

In [35]:
(define (good-enough? guess lastguess)
  (< (/ ( abs(- guess lastguess)) guess)
     0.001))

(define (improve guess x)
  (average guess (/ x guess)))

(define (average x y)
  (/ (+ x y) 2))

(define (sq-root guess x lastguess)
  (if (good-enough? guess lastguess)
      guess
      (sq-root(improve guess x)
             x
             guess)))
  
(sq-root 2 0.1 1)

.31622776688213083

Notes on this solution:
* v inexact. some of it may be from floating point math, plus it's only an approximation
* how do we pick a better starting point?
* where should state be stored? I'm using args rn, but it feels inelegant..

## 1.8

for y ~= x^(-1/3) (or at least a guess),  (x/(y^2) + 2y)/3 improves your guess for a cube root of x. do a cube root calculator

In [43]:
(define (improve guess target)
  (/(+(/ target (sq guess)) 
      (* 2 guess))
    3))

(define (sq x)
  (* x x))

(define (cube-root guess target lastguess)
  (if (good-enough? guess lastguess)
      guess
      (cube-root(improve guess target) target guess)))

(cube-root 1.0 8 8)

2.000000000012062

Notes:
* a lot of this is generalizeable from the sqrt method!
* the main sq-root method itself is actually a meta-strategy for estimation in general
* we can estimate anything if we have 
    1. a starting point
    2. a method for getting closer to that target
    3. a definition for when we're close enough to the target

In sqrt, our target was the value of the thing we're trying to find a sqrt for. Our method for getting closer was averaging `x` and `x/guess` to get the next `guess` value, and we used `good-enough?` to determine when we were close enough. All we have to do is provide our methods as args to the guesser, and we're good!